In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import math # for factorials

In [2]:
#df = pd.read_csv('./SPREAD_food_networks/sn/results/networks/1.edges', sep=',', usecols = ['source', 'target', 'month'])
df = pd.read_csv('graph.txt', sep = ' ')
def dup_edges(graph_df, delta, period = 12):
    """Duplicate each edge with new timestamps i > period+i""" 
    """CHANGE: now only duplicate edges until period + (delta-1)"""
    # df containing only months to be duplicated (months < delta)
    dup_in_delta = (graph_df.loc[graph_df['month'] < delta])
    
    return pd.concat([graph_df, 
                      pd.concat([dup_in_delta.iloc[:,[0, 1]], 
                                 dup_in_delta.iloc[:,[2]]+period], axis=1)], 
                                 ignore_index=True)
# df = dup_edges(df)
print(f'{df.tail()}\n')
d = dup_edges(df, delta = 5)
print(d.tail())

   source  target  month
5       4       2      5
6       4       2      6
7       3       5      5
8       5       3      4
9       3       6      6

    source  target  month
10       1       4     13
11       4       2     14
12       2       4     15
13       2       4     16
14       5       3     16


In [3]:
G = nx.from_pandas_edgelist(df, 'source', 'target', edge_attr='month', create_using=nx.MultiDiGraph)
G = nx.convert_node_labels_to_integers(G, ordering='sorted', label_attribute = 'old_id')
nx.get_node_attributes(G, 'old_id')


{0: 1, 3: 4, 1: 2, 2: 3, 4: 5, 5: 6}

In [4]:
list(G.edges(data=True))

[(0, 3, {'month': 1}),
 (0, 3, {'month': 7}),
 (3, 1, {'month': 2}),
 (3, 1, {'month': 5}),
 (3, 1, {'month': 6}),
 (1, 3, {'month': 3}),
 (1, 3, {'month': 4}),
 (2, 4, {'month': 5}),
 (2, 5, {'month': 6}),
 (4, 2, {'month': 4})]

In [6]:
for e in G.edges(data=True):
    e[2]['delta'] = {}
list(G.edges(data=True))

[(0, 3, {'month': 1, 'delta': {}}),
 (0, 3, {'month': 7, 'delta': {}}),
 (3, 1, {'month': 2, 'delta': {}}),
 (3, 1, {'month': 5, 'delta': {}}),
 (3, 1, {'month': 6, 'delta': {}}),
 (1, 3, {'month': 3, 'delta': {}}),
 (1, 3, {'month': 4, 'delta': {}}),
 (2, 4, {'month': 5, 'delta': {}}),
 (2, 5, {'month': 6, 'delta': {}}),
 (4, 2, {'month': 4, 'delta': {}})]

In [19]:
# create and fill the data structure for post processing
struc = {'t1': [], 't2': [], 'count': []}
# s choose 2 + s, in this case s=7
num_pairs = 7 + math.factorial(7)/(2 * math.factorial(7-2))
for i in range(int(num_pairs)):
    struc['count'].append(0)
for i in range(1, 8):
    j = i
    while j < 8:
        struc['t1'].append(i)
        struc['t2'].append(j)
        j += 1
print(struc)

{'t1': [1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 6, 6, 7], 't2': [1, 2, 3, 4, 5, 6, 7, 2, 3, 4, 5, 6, 7, 3, 4, 5, 6, 7, 4, 5, 6, 7, 5, 6, 7, 6, 7, 7], 'count': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [12]:
def bidirecCountAndStore(graph, delta):
    """
    graph: a Time-Varying directed network.
    delta: time-interval within which bidirectional edges must be counted.
    for each edge with timestamp t, computes the number of reverse edges within t + delta and stores this in
    its edge attribute.
    Returns the graph with the additional edge attribute
    """
    
    edges = sorted(graph.edges(data=True), key=lambda t: t[2].get('month', 1))
    bidirec_counts = []
    for i in range(len(edges)):
        count = 0
        j = i# + 1
        while j < len(edges):
            if edges[j][2]['month'] > edges[i][2]['month'] + delta:
                break
            elif edges[j][0] == edges[i][1] and edges[j][1] == edges[i][0]:
                count += 1
            j += 1
        edges[i][2]['delta'][str(delta)] = count
        bidirec_counts.append(count)
    return edges

In [13]:
G.edges(data=True)
edges = bidirecCountAndStore(G, delta = 4)
# relabel with original node ids
bidirecCountAndStore(G, delta = 2)

[(0, 3, {'month': 1, 'delta': {'4': 0, '2': 0}}),
 (3, 1, {'month': 2, 'delta': {'4': 2, '2': 2}}),
 (1, 3, {'month': 3, 'delta': {'4': 2, '2': 1}}),
 (1, 3, {'month': 4, 'delta': {'4': 2, '2': 2}}),
 (4, 2, {'month': 4, 'delta': {'4': 1, '2': 1}}),
 (3, 1, {'month': 5, 'delta': {'4': 0, '2': 0}}),
 (2, 4, {'month': 5, 'delta': {'4': 0, '2': 0}}),
 (3, 1, {'month': 6, 'delta': {'4': 0, '2': 0}}),
 (2, 5, {'month': 6, 'delta': {'4': 0, '2': 0}}),
 (0, 3, {'month': 7, 'delta': {'4': 0, '2': 0}})]

In [9]:
G = nx.relabel_nodes(G, nx.get_node_attributes(G, 'old_id'), copy=True)
list(G.edges(data=True))

[(1, 4, {'month': 1, 'delta': {'4': 0, '2': 0}}),
 (1, 4, {'month': 7, 'delta': {'4': 0, '2': 0}}),
 (4, 2, {'month': 2, 'delta': {'4': 2, '2': 2}}),
 (4, 2, {'month': 5, 'delta': {'4': 0, '2': 0}}),
 (4, 2, {'month': 6, 'delta': {'4': 0, '2': 0}}),
 (2, 4, {'month': 3, 'delta': {'4': 2, '2': 1}}),
 (2, 4, {'month': 4, 'delta': {'4': 2, '2': 2}}),
 (3, 5, {'month': 5, 'delta': {'4': 0, '2': 0}}),
 (3, 6, {'month': 6, 'delta': {'4': 0, '2': 0}}),
 (5, 3, {'month': 4, 'delta': {'4': 1, '2': 1}})]

In [9]:
def get_bidirec_counts(graph, delta, node_pair):
    """
    graph: MultiDiGraph with edge attribute 'delta' and key delta.
    delta: time interval
    node_pair: tuple: check birec edge counts between node pair
    returns total number of bidirectional edges between node pair within given delta.
    """
    count = 0
    for e in graph.edges(data=True):
        if (e[0] == node_pair[0] and e[1] == node_pair[1]) or (e[0] == node_pair[1] and e[1] == node_pair[0]):
            count += e[2]['delta'][str(delta)]
    return count

In [10]:
get_bidirec_counts(G, 4, (2, 4))

1

In [11]:
help(get_bidirec_counts)

Help on function get_bidirec_counts in module __main__:

get_bidirec_counts(graph, delta, node_pair)
    graph: MultiDiGraph with edge attribute 'delta' and key delta.
    delta: time interval
    node_pair: tuple: check birec edge counts between node pair
    returns total number of bidirectional edges between node pair within given delta.

